In [8]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'personal_projects/BgsCollection'))
	print(os.getcwd())
except:
	pass


 # Building dataframe for analysis

In [9]:
import pandas as pd
import sys
from BGG import BGG

bgg = BGG()

d = bgg.fetch_collection('scaroni')

allGamesDF = pd.DataFrame(d).drop_duplicates(subset=['name']).reindex()

allGamesDF['played'] = allGamesDF['rating'].map(lambda x: not x == None, na_action = None)

#remove expansions
allGamesDF['isExpansion'] = allGamesDF['categories'].map(lambda x: 'Expansion for Base-game' in x, na_action=None)
allGamesDF = allGamesDF[allGamesDF.isExpansion == False]

playedGamesDF = allGamesDF[allGamesDF.played == True]

# exclude not owned games and games with same name (multiple copies of same game)
ownedGamesDF = allGamesDF[(allGamesDF['owned'] == True)].drop_duplicates(subset=['name']).reindex()

def convertToInt(number):
    try:
        return int(number)
    except:
        return sys.maxsize

ownedGamesDF['bggRank'] = ownedGamesDF['bggRank'].map(convertToInt, na_action=None)

totalGamesCount = ownedGamesDF.name.size

print('Total games:' + str(totalGamesCount) + "\n")
print(ownedGamesDF.to_csv(columns=['name'], sep='\t', index=False, header=False))


177590
Found multiple entries for ./thumbnail, ignoring all but first
Found multiple entries for ./image, ignoring all but first
Found multiple entries for ./description, ignoring all but first
68448
Found multiple entries for ./thumbnail, ignoring all but first
Found multiple entries for ./image, ignoring all but first
Found multiple entries for ./description, ignoring all but first
8815
Found multiple entries for ./thumbnail, ignoring all but first
Found multiple entries for ./image, ignoring all but first
Found multiple entries for ./description, ignoring all but first
8815
Found multiple entries for ./thumbnail, ignoring all but first
Found multiple entries for ./image, ignoring all but first
Found multiple entries for ./description, ignoring all but first
17329
Found multiple entries for ./thumbnail, ignoring all but first
Found multiple entries for ./image, ignoring all but first
Found multiple entries for ./description, ignoring all but first
230802
Found multiple entries for ./

NameError: name 'd' is not defined

 # Counting Games per player capacity

In [10]:
s = ownedGamesDF.minplayers.value_counts().sort_values(ascending=False)
pd.Series(s.index.values, index=s ).plot.bar()



NameError: name 'ownedGamesDF' is not defined

In [6]:
s = ownedGamesDF.maxplayers.value_counts().sort_values(ascending=False)
pd.Series(s.index.values, index=s ).plot.bar()


NameError: name 'ownedGamesDF' is not defined

 # Analysing played games by rating

In [0]:
import matplotlib.pyplot as plt

notOwned = len(playedGamesDF[playedGamesDF.owned == False])
owned = len(playedGamesDF[playedGamesDF.owned == True])

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = ['Not currently owned: %s' % notOwned, 'Owned: %s' % owned]
sizes = [notOwned, owned]
colors = ['lightgreen', 'lightblue']
explode = (0, 0)

fig1, ax1 = plt.subplots()

ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, colors=colors)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

print ("All games played: How many i own, how many i played from others?")
plt.show()



In [0]:
print ("Played games: ")
playedGamesDF[['name', 'rating', 'owned']].style.set_properties(**{'text-align': 'left'})



In [0]:
unplayed = len(ownedGamesDF[ownedGamesDF.played == False])
played = len(ownedGamesDF[ownedGamesDF.played == True])

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = ['Played: %s' % played, 'Unplayed: %s' % unplayed]
sizes = [played, unplayed]
colors = ['yellowgreen', 'red']
explode = (0, 0.1)  # only "explode" the 2nd slice (i.e. 'unplayed')

fig1, ax1 = plt.subplots()

ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90, colors=colors)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

print ("How many games from my collection i have played and how many i have not?")

plt.show()



In [0]:
s = ownedGamesDF.dropna(subset=['rating']).rating.astype('float64').value_counts().sort_values(ascending=False)

pd.Series(s.index.values, index=s ).plot.bar()


 # Other analysis on games characteristics

In [0]:
targetdf = ownedGamesDF[ownedGamesDF.year != "None"][['name', 'year']]
targetdf['year'] = targetdf['year'].astype('int32')
bins = [1933., 1941.4, 1949.8, 1958.2, 1966.6, 1975.0, 1983.4, 1991.8, 2000.2, 2008.6, 2017.0 ]
print('games release date\n')
targetdf.hist(bins=bins)



In [0]:
# __slots__ = [
#     'designers', 'artists', 'playingtime', 'thumbnail',
#     'image', 'description', 'minplayers', 'maxplayers',
#     'categories', 'mechanics', 'families', 'publishers',
#     'website', 'year', 'names', 'bgid'
# ]

allCategories = {}

for i in ownedGamesDF.index:
    for cat in ownedGamesDF.loc[i].categories:
        if cat in allCategories:
            allCategories[cat].append(ownedGamesDF.loc[i]['name'])
        else:
            allCategories[cat] = [ownedGamesDF.loc[i]['name']]

categories = [(tup[0], len(tup[1])) for tup in sorted(list(allCategories.items()), key=lambda tup: tup[1], reverse=True)[:10]]
count = [x[1] for x in categories]

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = categories
sizes = count

fig1, ax1 = plt.subplots()

ax1.pie(sizes, labels=labels,
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()



In [0]:
# 'Bluffing', 'Humor', 'Party Game', 'Economic', 'Negotiation', 'American West',
# 'Card Game', 'Deduction', 'Fighting', 'Industry / Manufacturing', 'Medieval',
# 'Puzzle', 'Adventure', 'Fantasy', 'Miniatures', 'Novel-based', 'Wargame', 'Horror',
# 'Murder/Mystery', 'Dice', 'Mythology', 'Comic Book / Strip', 'Movies / TV / Radio theme',
# 'Number', "Children's Game", 'Farming', 'Print & Play', 'Science Fiction', 'Exploration',
# 'Territory Building', 'Prehistoric', 'Zombies', 'Renaissance', ' ', '-',
# 'Collectible Components', 'City Building', 'Maze', 'Real-time', 'Trivia', 'Medical',
# 'Abstract Strategy', 'Space Exploration', 'Action / Dexterity', 'Spies/Secret Agents',
# 'Mafia'

cat1 = ['Bluffing', 'Humor', 'Party Game']
cat2 = ['Farming', 'Economic', 'City Building', 'Abstract Strategy', 'Territory Building']
cat3 = ['Card Game']
cat4 = ['Space Exploration', 'Adventure', 'Fantasy', 'Exploration']

categories = [cat1, cat2, cat3, cat4]

for cat in categories:
    games = set()
    for c in cat:
        games.update(allCategories[c])
    games = list(games)
    top10 = pd.DataFrame(data={'games':games})
    print(top10.to_string(justify='left', header=False), "\n")





In [0]:
a = [1,2,3,4]
b = [4,5,6, 1]

s = set(a)
s.update(b)
print(s)



In [0]:
allMechanicsCount = {}

for i in ownedGamesDF.index:
  for mechanic in ownedGamesDF.loc[i].mechanics:
    if not mechanic == '-' and not mechanic == ' ':
      if mechanic in allMechanicsCount:
        allMechanicsCount[mechanic] = allMechanicsCount[mechanic] + 1
      else:
        allMechanicsCount[mechanic] = 1

mechanics = sorted(list(allMechanicsCount.items()), key=lambda tup: tup[1], reverse=True)[:10]
count = [x[1] for x in mechanics]

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = mechanics
sizes = count

fig1, ax1 = plt.subplots()

ax1.pie(sizes, labels=labels,
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()



In [0]:
import sys

allMechanicsCount = {}
mechanicsAndGames = {}

for i in ownedGamesDF.index:
  for mechanic in ownedGamesDF.loc[i].mechanics:
    if not mechanic == '-' and not mechanic == ' ':
      if mechanic in allMechanicsCount:
        allMechanicsCount[mechanic] = allMechanicsCount[mechanic] + 1
        mechanicsAndGames[mechanic].append(ownedGamesDF.loc[i]['name'])
      else:
        allMechanicsCount[mechanic] = 1
        mechanicsAndGames[mechanic] = [ownedGamesDF.loc[i]['name']]

def SortByBggRank(name):
  try:
    return int(ownedGamesDF[ownedGamesDF.name == name].bggRank)
  except:
    return sys.maxsize

Top10GamesPerMechanics = [(x[0], sorted(mechanicsAndGames[x[0]], key=SortByBggRank)) for x in sorted(list(allMechanicsCount.items()), key=lambda tup: tup[1], reverse=True)[:10]]


for mechanic in Top10GamesPerMechanics:
  print(mechanic[0], ":")
  top10 = pd.DataFrame(data={'games':mechanic[1]})
  print(top10.to_string(justify='left', header=False), "\n")




In [0]:
designersCount = {}

for i in ownedGamesDF.index:
  if ownedGamesDF.loc[i].isExpansion or ownedGamesDF.loc[i].designers == ' - ' or '(Uncredited)' in ownedGamesDF.loc[i].designers:
    continue
  for designer in ownedGamesDF.loc[i].designers:
    if designer in designersCount:
      designersCount[designer] = designersCount[designer] + 1
    else:
      designersCount[designer] = 1

data = list(designersCount.items())
data.sort(key=lambda tup: tup[0])  # sorts in place
data.sort(key=lambda tup: tup[1], reverse=True)  # sorts in place

d = {'designer': [x[0] for x in data[:10]], 'number of games': [x[1] for x in data[:10]]}
top10 = pd.DataFrame(data=d)
top10.style.set_properties(**{'text-align': 'center'})



In [0]:
ownedGamesDF.sort_values(by=['bggRank'])[['name', 'bggRank']].head(11).style.set_properties(**{'text-align': 'center'})



In [0]:
# my top 10 owned

ownedGamesDF.sort_values(by=['rating'], ascending=False)[['name', 'rating']].head(11).style.set_properties(**{'text-align': 'center'})



In [0]:
# my top 10 played
playedGamesDF.sort_values(by=['rating'], ascending=False)[['name', 'rating']].head(11).style.set_properties(**{'text-align': 'center'})



In [0]:
print(playedGamesDF.to_csv(columns=['name', 'rating'], sep=' ', index=False, header=False))



In [0]:
ownedGamesDF[ownedGamesDF.bggRank < sys.maxsize].sort_values(by=['bggRank'], ascending=False)[['name', 'bggRank']].head(10).style.set_properties(**{'text-align': 'center'})



In [0]:
targetdf = ownedGamesDF[['name', 'weight']]
targetdf['weight'] = targetdf['weight'].astype('float64')
print('weight distribution\n')
targetdf.hist()



In [0]:
print ("colection avarage weight: ", ownedGamesDF.weight.mean())



In [0]:
lightGames = ownedGamesDF.weight < 1.6
mediumLightGames = (ownedGamesDF.weight < 2.1) & (ownedGamesDF.weight >= 1.6)
mediumGames = (ownedGamesDF.weight < 2.6) & (ownedGamesDF.weight >= 2.1)
mediumHeavyGames = (ownedGamesDF.weight < 3.6) & (ownedGamesDF.weight >= 2.6)
heavyGames = (ownedGamesDF.weight >= 3.6)

print ("Light games")
print (ownedGamesDF[lightGames][["name", "weight"]].sort_values("weight", ascending=False))
print ("\nMedium-light games")
print (ownedGamesDF[mediumLightGames][["name", "weight"]].sort_values("weight", ascending=False))
print ("\nMedium games")
print (ownedGamesDF[mediumGames][["name", "weight"]].sort_values("weight", ascending=False))
print ("\nMedium-heavy games")
print (ownedGamesDF[mediumHeavyGames][["name", "weight"]].sort_values("weight", ascending=False))
print ("\nHeavy games")
print (ownedGamesDF[heavyGames][["name", "weight"]].sort_values("weight", ascending=False))



In [0]:
ownedGamesDF[["name", "description"]]


